In [ ]:
class LBO_Model:
    """
    LBO Model for estimating the exit price of a company using a probabilistic approach and sensitivity analysis
    """
    def __init__(self, purchase_price:float, debt:float, interest_rate:float, equity:float, revenue:float, growth_rate_mean:float, growth_rate_stddev:float, operating_margin:float, capital_expenditures:List[float], working_capital:List[float], exit_year:int, non_operating_expenses:List[float], inflation_rate:float, tax_rate:float, debt_issuance:List[float], equity_issuance:List[float], exit_multiple_mean:float, exit_multiple_stddev:float, num_samples:int):
        """
        Initialize the variables
        :param purchase_price: Purchase price of the company
        :param debt: Initial debt of the company
        :param interest_rate: Interest rate on the debt
        :param equity: Initial equity of the company
        :param revenue: Initial revenue of the company
        :param growth_rate_mean: Mean of the revenue growth rate
        :param growth_rate_stddev: Standard deviation of the revenue growth rate
        :param operating_margin: Operating margin of the company
        :param capital_expenditures: List of capital expenditures for each year
        :param working_capital: List of working capital for each year
        :param exit_year: Number of years until exit
        :param non_operating_expenses: List of non-operating expenses for each year
        :param inflation_rate: Inflation rate for each year
        :param tax_rate: Tax rate for the company
        :param debt_issuance: List of debt issuance for each year
        :param equity_issuance: List of equity issuance for each year
        :param exit_multiple_mean: Mean of the exit multiple
        :param exit_multiple_stddev: Standard deviation of the exit multiple
        :param num_samples: Number of samples for the probabilistic approach
        """
        self.purchase_price = purchase_price
        self.debt = debt
        self.interest_rate = interest_rate
        self.equity = equity
        self.revenue = revenue
        self.growth_rate_mean = growth_rate_mean
        self.growth_rate_stddev = growth_rate_stddev
        self.operating_margin = operating_margin
        self.capital_expenditures = capital_expenditures
        self.working_capital = working_capital
        self.exit_year = exit_year
        self.non_operating_expenses = non_operating_expenses
        self.inflation_rate = inflation_rate
        self.tax_rate = tax_rate
        self.debt_issuance = debt_issuance
        self.equity_issuance = equity_issuance
        self.exit_multiple_mean = exit_multiple_mean
        self.exit_multiple_stddev = exit_multiple_stddev
        self.num_samples = num_samples
        self.growth_rate_distribution = stats.norm(self.growth_rate_mean, self.growth_rate_stddev)
        self.exit_multiple_distribution = stats.norm(self.exit_multiple_mean, self.exit_multiple_stddev)
        self.project_revenue()
        self.project_operating_income()
        self.project_capital_structure()
        self.project_net_income()
        self.calculate_cash

    def project_revenue(self):
        """
        Project the revenue for each year using the growth rate
        """
        self.revenue_growth_rate = self.growth_rate_distribution.rvs(self.exit_year)
        self.revenue_projection = [self.revenue]
        for i in range(1, self.exit_year):
            self.revenue_projection.append(self.revenue_projection[i - 1] * (1 + self.revenue_growth_rate[i]))

    def project_operating_income(self):
        """
        Project the operating income for each year using the revenue and operating margin
        """
        self.operating_income = [(x * self.operating_margin) for x in self.revenue_projection]

    def project_capital_structure(self):
        """
        Project the capital structure for each year using the debt, equity, debt issuance and equity issuance
        """
        self.debt_projection = [self.debt]
        self.equity_projection = [self.equity]
        for i in range(1, self.exit_year):
            self.debt_projection.append(self.debt_projection[i - 1] + self.debt_issuance[i])
            self.equity_projection.append(self.equity_projection[i - 1] + self.equity_issuance[i])
        self.capital_structure_projection = [x + y for x, y in zip(self.debt_projection, self.equity_projection)]

    def project_net_income(self):
        """
        Project the net income for each year using the operating income, capital expenditures, working capital, non-operating expenses, inflation and tax rate
        """
        self.net_income = []
        for i in range(self.exit_year):
            self.net_income.append(self.operating_income[i] - self.capital_expenditures[i] - self.working_capital[i] - self.non_operating_expenses[i])
        self.net_income = [(x * (1 - self.tax_rate)) for x in self.net_income]

    def calculate_cash(self):
        """
        Calculate the cash for each year using the net income and capital structure
        """
        self.cash = []
        for i in range(self.exit_year):
            self.cash.append(self.net_income[i] + (self.debt_projection[i] - self.debt_projection[i-1])* self.interest_rate - (self.equity_projection[i] - self.equity_projection[i-1]))
    
    def calculate_exit_price(self):
        """
        Calculate the exit price using the cash and exit multiple
        """
        self.exit_multiple = self.exit_multiple_distribution.rvs(self.num_samples)
        self.exit_price = [x * y for x, y in zip(self.cash, self.exit_multiple)]
        self.exit_price_mean = np.mean(self.exit_price)
        self.exit_price_stddev = np.std(self.exit_price)
        
    def create_sensitivity_analysis(self):
        """
        Create a sensitivity analysis of the exit price to different variables
        """
        self.sensitivity_data = {'Variable': [], 'Mean': [], 'Std Dev': []}
        self.sensitivity_data['Variable'].append('Operating Margin')
        self.sensitivity_data['Mean'].append(self.operating_margin)
        self.sensitivity_data['Std Dev'].append(0)
        self.sensitivity_data['Variable'].append('Capital Expenditures')
        self.sensitivity_data['Mean'].append(np.mean(self.capital_expenditures))
        self.sensitivity_data['Std Dev'].append(np.std(self.capital_expenditures))
        self.sensitivity_data['Variable'].append('Working Capital')
        self.sensitivity_data['Mean'].append(np.mean(self.working_capital))
        self.sensitivity_data['Std Dev'].append(np.std(self.working_capital))
        self.sensitivity_data['Variable'].append('Non-Operating Expenses')
        self.sensitivity_data['Mean'].append(np.mean(self.non_operating_expenses))
        self.sensitivity_data['Std Dev'].append(np.std(self.non_operating_expenses))
        self.sensitivity_data['Variable'].append('Inflation Rate')
        self.sensitivity_data['Mean'].append(self.inflation_rate)
        self.sensitivity_data['Std Dev'].append(0)
        self.sensitivity_data['Variable'].append('Tax Rate')
        self.sensitivity_data['Mean'].append(self.tax_rate)
        self.sensitivity_data['Std Dev'].append(0)
        self.sensitivity_data['Variable'].append('Debt Issuance')
        self.sensitivity_data['Mean'].append(np.mean(self.debt_issuance))
        self.sensitivity_data['Std Dev'].append(np.std(self.debt_issuance))
        self.sensitivity_data['Variable'].append('Equity Issuance')
        self.sensitivity_data['Mean'].append(np.mean(self.equity_issuance))
        self.sensitivity_data['Std Dev'].append(np.std(self.equity_issuance))
        self.sensitivity_data['Variable'].append('Exit Multiple')
        self.sensitivity_data['Mean'].append(self.exit_multiple_mean)
        self.sensitivity_data['Std Dev'].append(self.exit_multiple_stddev)
        self.sensitivity_data = pd.DataFrame(self.sensitivity_data)
        
    def create_tornado_diagram(self):
        """
        Create a tornado diagram to show the sensitivity of the exit price to different variables
        """
        self.sensitivity_data['Lower Bound'] = self.sensitivity_data['Mean'] - 2 * self.sensitivity_data['Std Dev']
        self.sensitivity_data['Upper Bound'] = self.sensitivity_data['Mean'] + 2 * self.sensitivity_data['Std Dev']
        self.sensitivity_data['Exit Price'] = self.exit_price_mean
        self.sensitivity_data = self.sensitivity_data.sort_values(by='Exit Price', ascending=False)
        plt.figure(figsize=(10, 6))
        plt.barh(y=self.sensitivity_data['Variable'], width=self.sensitivity_data['Exit Price'], height=0.4, left=self.sensitivity_data['Lower Bound'], align='center', color='skyblue')
        plt.barh(y=self.sensitivity_data['Variable'], width=-(self.sensitivity_data['Upper Bound']-self.sensitivity_data['Lower Bound']), height=0.4, left=self.sensitivity_data['Upper Bound'], align='center', color='white')
        plt.xlabel('Exit Price')
        plt.title('Tornado Diagram')
        plt.show()
        
    def run_model(self):
        """
        Run the LBO model and print the results
        """
        self.calculate_exit_price()
        self.create_sensitivity_analysis()
        self.create_tornado_diagram()
        print("Exit Price Mean: ", self.exit_price_mean)
        print("Exit Price Standard Deviation: ", self.exit_price_stddev)

In [ ]:
# initialize variables
purchase_price = 100
debt = 50
interest_rate = 0.1
equity = 50
revenue = 100
growth_rate_mean = 0.05
growth_rate_stddev = 0.02
operating_margin = 0.2
capital_expenditures = [5, 5, 5]
working_capital = [10, 10, 10]
exit_year = 3
non_operating_expenses = [5, 5, 5]
inflation_rate = 0.03
tax_rate = 0.2
debt_issuance = [5, 5, 5]
equity_issuance = [5, 5, 5]
exit_multiple_mean = 8
exit_multiple_stddev = 2
num_samples = 1000

# create LBO model
lbo_model = LBOModel(purchase_price, debt, interest_rate, equity, revenue, growth_rate_mean, growth_rate_stddev, operating_margin, capital_expenditures, working_capital, exit_year, non_operating_expenses, inflation_rate, tax_rate, debt_issuance, equity_issuance, exit_multiple_mean, exit_multiple_stddev, num_samples)
lbo_model.run_model()